# Skip-gram word2vec

在这个 notebook 里，你将使用 TensorFlow 去实现基于 skip-gram 架构的 word2vec 模型。通过实现这一点，你将学习到单词嵌入在自然语言处理中的应用。 这在处理机器翻译等事情时会派上用场。

## 资源

这些是可以阅读的资源。建议你在开始学习之前先阅读这些资料或者边学边看以加深你的理解。


* Chris McCormick 写的很不错的 [word2vec 概念介绍](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)  
* Mikolov 等人的[第一篇关于 word2vec 的论文](https://arxiv.org/pdf/1301.3781.pdf)
* Mikolov 等人对 word2vec 的改进的[ NIPS 论文](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)
* Thushan Ganegedara 的 [ word2vec 实现案例](http://www.thushv.com/natural_language_processing/word2vec-part-1-nlp-with-deep-learning-with-tensorflow-skip-gram/) 
* TensorFlow文献 [word2vec 教程](https://www.tensorflow.org/tutorials/word2vec)


## 词嵌入

当你处理文本中的单词时，每一个单词都将需要数以万计的类来预测。尝试对这些单词进行独热编码处理是非常低效的，你需要将一个元素设置为 1 而其它的 50000 个元素设置为 0 .进入第一个隐藏层的矩阵运算的大多数乘积都会是 0 。 这极大地浪费了计算量，很没有效率。

![独热编码](assets/one_hot_encoding.png)

为了解决这个问题并大大提高我们网络的效率，我们引入了嵌入。嵌入只是一个完全连接的层，就像你以前见过的那样。我们将此图层称为嵌入图层，权重称为嵌入权重。 我们通过直接从权重矩阵中抓取隐藏层的值来跳过矩阵相乘的过程。我们之所以可以这样做，是因为我们是用矩阵和一个独热编码向量相乘，返回与“on”输入单元的索引相对应的矩阵行。

![查询表](assets/lookup_matrix.png)

我们使用权重矩阵作为查询表来代替矩阵乘法。我们将单词编码为整数，例如“heart”被编码为958，“mind”被编码为18094.然后，为了获得“heart”的隐藏层值，只需获取嵌入矩阵的第958行。 此过程称为**嵌入查询表**，隐藏的单元数称为**嵌入维数** 。

<img src='assets/tokenize_lookup.png' width=500>
 
所以嵌入其实没什么神奇的。嵌入查询表其实就是权重矩阵。嵌入层就是隐藏层。这种查询其实是矩阵相乘的捷径。由于查询表就是权重矩阵，所以你可以像处理之前遇到的其它权重矩阵一样训练它。

嵌入不仅可用来处理单词，你也可以用它来处理任何具有庞大类的模型。我们这里要讲一种叫做 **word2vec**的模型，该模型用嵌入层来从这些包含词语义的向量中获取这类词表征。

## Word2Vec

word2vec 算法会用嵌入层的向量来表示单词，从而实现更佳的表征方式。这些向量还包含有关单词的语义信息。处于类似语境的单词，例如黑色，白色和红色，其对应的向量值会很接近。word2vec 有两种实现结构，一种是 CBOW（连续词袋），一种是 Skip-gram。

<img src="assets/word2vec_architectures.png" width="500">

在这个实现中，我们将使用 skip-gram 架构，因为它比CBOW表现要好。在这里，我们输入一个单词并试着预测这个词在文本里的语境。通过这种方式，我们可以训练网络来学习在类似语境中出现的单词的词义。

首先，导入要用的包。

In [ ]:
import time

import numpy as np
import tensorflow as tf

import utils

加载 [text8 数据集](http://mattmahoney.net/dc/textdata.html), 这是一批来自 Matt Mahoney 的已经清理好的维基百科的文章。下一个单元我们会将数据集下载到`data`文件夹。 之后，你可以提取它并删除存档文件以节省存储空间。

In [ ]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm
import zipfile

dataset_folder_path = 'data'
dataset_filename = 'text8.zip'
dataset_name = 'Text8 Dataset'

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile(dataset_filename):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc=dataset_name) as pbar:
        urlretrieve(
            'http://mattmahoney.net/dc/text8.zip',
            dataset_filename,
            pbar.hook)

if not isdir(dataset_folder_path):
    with zipfile.ZipFile(dataset_filename) as zip_ref:
        zip_ref.extractall(dataset_folder_path)
        
with open('data/text8') as f:
    text = f.read()

## 预处理

为使训练更容易，我对文本做了一些预处理。这些操作我都写在 utils 模块里了。 `preprocess`函数用于将标点符号转换为令牌（tokens），因此句号变成了 `<PERIOD>`。在这个数据集中，没有句号，但它将有助于其他 NLP 问题。我也删除了在数据集中出现次数小于或等于五次的单词。这将大大减少由于数据中的噪声引发的问题并提高向量表征的质量。你也可以为这些东西编写自己的函数。

In [ ]:
words = utils.preprocess(text)
print(words[:30])

In [ ]:
print("Total words: {}".format(len(words)))
print("Unique words: {}".format(len(set(words))))

然后创建两个字典，一个把单词转为整数，一个把整数转回单词。整数按单词出现的频率降序分配，因此最常用的字（“the”）分配整数 0 ，下一个常用的单词分配整数 1 ，依此类推。因此，单词被转换为整数并存储在列表`int_words`中。

In [ ]:
vocab_to_int, int_to_vocab = utils.create_lookup_tables(words)
int_words = [vocab_to_int[word] for word in words]

## 子采样

比如 the、of、and 和 for 这种单词出现频率很高，但不能为周围的单词提供什么语境信息，所以我们可以直接删除这类高频词，这样就能减少数据噪声，提高训练率，最后得到的表征效果也更好。这种舍弃高频单词的过程叫做子采样。对于训练集中的每个单词 $w_i$ ，我们要计算其出现的频率 $f(w_i)$ ，从而获得一个舍弃每个高频单词的概率

 $$ P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}} $$

 其中 $t$ 是阈值参数，能让我们设定阀值，也就是设定什么频率的单词会被以某种概率舍弃。

接下来的练习就交给你了。虽然这更像是编程挑战，而不像实际的深度学习，但对你获得数据准备的经验是很重要的。可以通过我的解决方案，看我是如何做到的。

> **练习：** 你需要实现子采样，处理`int_words`里的单词。思路就是遍历数据集`int_words`，以上面给定的概率 $P(w_i)$ 舍弃每个单词（注意 $P(w_i)$ 是一个单词被舍弃的概率）。然后将子采样的数据结果赋给数据集`train_words`。

In [ ]:
## Your code here
train_words = # The final subsampled word list

## 进行分批

现在我们的数据状况良好，我们需要将其转换为适当的形式以将其传递到我们的网络中。使用 skip-gram 结构处理每个单词的时候，我们要抓取一个由输入词周围单词组成的窗口，这个窗口的大小为 $C$。 

[Mikolov 等人的文章](https://arxiv.org/pdf/1301.3781.pdf): 

“因为远些的单词往往与输入单词关系较浅，而近些的单词则较深，因此我们通过在训练示例中对这些单词进行低采样来减少相对较远单词的权重...假设我们的窗口大小为 $C$ 且 $C = 5$ ，在 1 到 C 之间选一个随机数 R ，相当于你随机在大窗口里选了一个由 R 定义的小窗口，在小窗口里 你要以在输入单词前和后的 R ”

> **练习：** 在函数 `get_target` 里实现获取目标单词这些操作，你要输入一串单词，一个索引和一个窗口大小，然后在索引周围抓取一个窗口，并返回窗口里的所有目标单词。确保使用上述算法，从窗口中选择随机数量的单词。


In [ ]:
def get_target(words, idx, window_size=5):
    ''' Get a list of words in a window around an index. '''
    
    # Your code here
    
    return

这是要用的批次返回函数，也就是我们要传递给网络的数据。这个函数会从文本列表里抓取一些单词（`batch_size`），然后获取出现在窗口中的目标单词。我还没找到什么好办法，把目标单词的这个随机获取范围和 TensorFlow 图结合起来，所以这里我就只是在批次里设置一行一个输入目标对。这是一个生成函数，有助于节省内存。

In [ ]:
def get_batches(words, batch_size, window_size=5):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    
    n_batches = len(words)//batch_size
    
    # only full batches
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y
    

## 构建网络

这是[Chris McCormick 的博客](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)，我们可以看到网络的总体结构
![embedding_network](./assets/skip_gram_net_arch.png)

我们要输入的是整数。整数会被传递到隐藏层里，下一层是 softmax 输出层。我们会用这个 softmax 层来预测目标单词。

这里的想法是训练隐藏层的权重矩阵，得到包含语义的单词向量表征。训练好后，就可以把这个 softmax 层抛开了，因为我们并不真正关心用这个网络进行预测。我们只想要嵌入矩阵，以便我们可以在从数据集构建的其他网络中使用它。 

现在我们要分阶段构建网络。首先，首先，创建 `inputs` 和 `labels` 占位符。

> **练习：** 用 `tf.placeholder` 创建 `inputs` 和 `labels` 。我们要传入整数，因此将数据类型设置为`tf.int32`。因为我们要传递的批次大小随机，所以要将批大小设为随机，因而设为[`None`]。要让代码能实现后面的操作，你需要将 `labels` 的第二维度设置为 `None` 或 `1` 。

In [ ]:
train_graph = tf.Graph()
with train_graph.as_default():
    inputs = 
    labels = 

## 嵌入层



嵌入矩阵的大小是是词典单词数与所需嵌入维数的乘积。比如你有一万个单词，三百个隐藏单元（即嵌入维数），那矩阵大小就是 $10,000 \times 300$ 。记住，我们使用标记化数据作为输入，通常是整数，其中标记的数量是我们词汇表中的单词数。


> **练习：** Tensorflow 为我们提供了函数 [`tf.nn.embedding_lookup`](https://www.tensorflow.org/api_docs/python/tf/nn/embedding_lookup) 来进行查询。通过把嵌入矩阵和一些整数传递给函数，函数就会进行查询了，它会返回矩阵中对应于那些整数的行。接下来，你要设置嵌入特征数，即嵌入维数（200是一个好的开始），然后创建嵌入矩阵变量，并使用`tf.nn.embedding_lookup`来获取嵌入张量。 对于嵌入矩阵，我建议你使用随机分布的函数  [tf.random_uniform](https://www.tensorflow.org/api_docs/python/tf/random_uniform) ，将范围初始化为 -1 到 1 。

In [ ]:
n_vocab = len(int_to_vocab)
n_embedding =  # Number of embedding features 
with train_graph.as_default():
    embedding = # create embedding weight matrix here
    embed = # use tf.nn.embedding_lookup to get the hidden layer output

## 负采样



对于我们给网络的每个例子，我们使用 softmax 层的输出来训练它。这意味着对于每个输入，虽然权重有数百万个，但我们要变的东西很少，因为我们只有一个例子为真。这使得对网络的训练效率非常低。 我们可以通过仅一次更新所有权重的一小部分来估算 softmax 层的损失。虽然我们要更新权重，使其具备正确的标签，但我们只对一个小小的错误标签样本进行处理，这就叫["负采样"](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)。 TensorFlow 有函数可以实现这一点，那就是[`tf.nn.sampled_softmax_loss`](https://www.tensorflow.org/api_docs/python/tf/nn/sampled_softmax_loss)。

> **练习：** 接下来，为 softmax 层创建权重和偏置。然后用权重和偏置来算出损失，计算函数是 [`tf.nn.sampled_softmax_loss`](https://www.tensorflow.org/api_docs/python/tf/nn/sampled_softmax_loss)。请务必阅读文档以了解其工作原理。

In [ ]:
# Number of negative labels to sample
n_sampled = 100
with train_graph.as_default():
    softmax_w = # create softmax weight matrix here
    softmax_b = # create softmax biases here
    
    # Calculate the loss using negative sampling
    loss = tf.nn.sampled_softmax_loss 
    
    cost = tf.reduce_mean(loss)
    optimizer = tf.train.AdamOptimizer().minimize(cost)

## 验证

此代码来自 Thushan Ganegedara 。 在这里，我们将选择几个常用词和几个不常见的词。 然后，我们输出最接近它们的单词。这是一种很好的方法去检查我们的嵌入表是否将具有相似语义含义的单词组合在一起。

In [ ]:
with train_graph.as_default():
    ## From Thushan Ganegedara's implementation
    valid_size = 16 # Random set of words to evaluate similarity on.
    valid_window = 100
    # pick 8 samples from (0,100) and (1000,1100) each ranges. lower id implies more frequent 
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples, 
                               random.sample(range(1000,1000+valid_window), valid_size//2))

    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keep_dims=True))
    normalized_embedding = embedding / norm
    valid_embedding = tf.nn.embedding_lookup(normalized_embedding, valid_dataset)
    similarity = tf.matmul(valid_embedding, tf.transpose(normalized_embedding))

In [ ]:
# If the checkpoints directory doesn't exist:
!mkdir checkpoints

## 训练

下面是训练网络的代码。每100批就输出一次训练损失。 每1000批次，它将输出验证字。

In [ ]:
epochs = 10
batch_size = 1000
window_size = 10

with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    iteration = 1
    loss = 0
    sess.run(tf.global_variables_initializer())

    for e in range(1, epochs+1):
        batches = get_batches(train_words, batch_size, window_size)
        start = time.time()
        for x, y in batches:
            
            feed = {inputs: x,
                    labels: np.array(y)[:, None]}
            train_loss, _ = sess.run([cost, optimizer], feed_dict=feed)
            
            loss += train_loss
            
            if iteration % 100 == 0: 
                end = time.time()
                print("Epoch {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Avg. Training loss: {:.4f}".format(loss/100),
                      "{:.4f} sec/batch".format((end-start)/100))
                loss = 0
                start = time.time()
            
            if iteration % 1000 == 0:
                ## From Thushan Ganegedara's implementation
                # note that this is expensive (~20% slowdown if computed every 500 steps)
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = int_to_vocab[valid_examples[i]]
                    top_k = 8 # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k+1]
                    log = 'Nearest to %s:' % valid_word
                    for k in range(top_k):
                        close_word = int_to_vocab[nearest[k]]
                        log = '%s %s,' % (log, close_word)
                    print(log)
            
            iteration += 1
    save_path = saver.save(sess, "checkpoints/text8.ckpt")
    embed_mat = sess.run(normalized_embedding)

如果需要，还原已训练的网络：

In [ ]:
with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    embed_mat = sess.run(embedding)

## 可视化单词向量

最后在这里我们用 T-SNE 来把向量归类分组，将这些表征在高维空间中可视化出来。T-SNE 用于将这些向量投影到二维，同时保留局部结构。查看 [来自 Christopher Olah 的这篇文章](http://colah.github.io/posts/2014-10-Visualizing-MNIST/) 来了解更多有关 T-SNE 和其他可视化高维数据的方法。

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [ ]:
viz_words = 500
tsne = TSNE()
embed_tsne = tsne.fit_transform(embed_mat[:viz_words, :])

In [ ]:
fig, ax = plt.subplots(figsize=(14, 14))
for idx in range(viz_words):
    plt.scatter(*embed_tsne[idx, :], color='steelblue')
    plt.annotate(int_to_vocab[idx], (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7)